In [1]:
search_for = 131
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.24977564811706543
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 296765305
type: [1, 1, 1, 1, 131] 131
cases of this type: 294499921
100000 5.411993321677585
200000 8.287340072508236
300000 8.476899744553522
400000 8.369215251331507
500000 10.450040216290637
600000 12.003065956819361
700000 12.430019433803054
800000 12.297828605448798
900000 12.300268720596621
1000000 14.043009271452725
1100000 14.056236341273282
1200000 8.090385965920941
1300000 19.82004093156283
1400000 21.578724083178127
1500000 21.883207205269915
1600000 21.47910833462125
1700000 22.80454097268232
1800000 22.016832703916666
1900000 21.73140821396719
2000000 21.927720497647663
2100000 21.96767458850191
2200000 22.06279443615695
2300000 5.767073398327456
2400000 24.797125120131355
2500000 27.45715772990835
2600000 27.135371469823994
2700000 26.920343799031563
2800000 26.590071437632172
2900000 27.162163448872896
3000000 26.12

29400000 21.33663078569986
29500000 7.532999892398
29600000 21.929390209608346
29700000 21.670319903434123
29800000 21.751604199124834
29900000 20.901055559183337
30000000 21.813811392999025
30100000 20.535511308279347
30200000 20.24853228217162
30300000 19.975031975555492
30400000 5.947796488893538
30500000 20.823045063226466
30600000 19.083962142843813
30700000 20.786621551500858
30800000 20.91918890560455
30900000 20.513663746543397
31000000 21.256419660015492
31100000 20.485921732704682
31200000 21.55603094840168
31300000 8.727896270189643
31400000 21.01297954837748
31500000 21.584086249888628
31600000 21.16585003104961
31700000 18.397902538265235
31800000 11.35635190568429
31900000 20.789783235048564
32000000 20.518385725471855
32100000 20.119979633416065
32200000 20.491892929970295
32300000 19.216033944167165
32400000 19.954582329212744
32500000 18.365660387364954
32600000 12.280052632906363
32700000 13.931118656117532
32800000 18.012234792443035
32900000 20.02492725159693
330000

59100000 18.642406700800837
59200000 18.541313805512488
59300000 17.64565051943131
59400000 16.759899821338177
59500000 18.29081562319416
59600000 7.400810838701069
59700000 16.07518456948105
59800000 17.88942624626847
59900000 17.187947145094174
60000000 18.43433960425497
60100000 17.44399427747783
60200000 19.74182877118693
60300000 8.437459340316405
60400000 19.734272881661692
60500000 20.15868278685376
60600000 19.0654700286582
60700000 20.786716683371562
60800000 19.60439268119073
60900000 20.48024631078265
61000000 20.095854943758635
61100000 19.33773537217737
61200000 8.537149261833266
61300000 17.046202814429055
61400000 19.397706572853206
61500000 17.840262337748644
61600000 18.03981391584046
61700000 18.043901435828715
61800000 15.06310042204944
61900000 8.833951030441941
62000000 17.077003935582017
62100000 18.218219386372123
62200000 17.489828197871923
62300000 17.708719577221657
62400000 18.93831538677916
62500000 9.861663677119791
62600000 18.003475246789364
62700000 18.9

88900000 10.6216761210714
89000000 15.226261632712953
89100000 14.904820915082528
89200000 15.44491058690702
89300000 6.478260472234252
89400000 15.57103212552135
89500000 15.557302226643017
89600000 16.253485298916296
89700000 16.535532270582504
89800000 16.409741029328526
89900000 16.802602989118142
90000000 16.609183146534956
90100000 16.30236150575324
90200000 16.814455442900833
90300000 16.242870788933423
90400000 15.957446297928502
90500000 15.126107121647578
90600000 12.745717341054974
90700000 8.573107445187539
90800000 14.675922991848651
90900000 14.517564249443268
91000000 15.204808512726258
91100000 4.334844225853194
91200000 15.327001929777524
91300000 15.04782445201535
91400000 14.253293324328366
91500000 8.448325185439701
91600000 13.130284240553944
91700000 14.794158515852741
91800000 15.956707419546417
91900000 15.752877589869408
92000000 16.35500146685622
92100000 16.098913393998224
92200000 16.46041572539279
92300000 16.5632222115879
92400000 16.262466501228367
925000

117900000 8.30905066022669
118000000 10.47142619126905
118100000 5.973055568776756
118200000 12.545872778975784
118300000 5.7103173423239895
118400000 13.07488113339946
118500000 12.615292336164979
118600000 12.73970226296103
118700000 13.032245474623165
118800000 12.68258798591527
118900000 13.159832907973676
119000000 13.464799788379397
119100000 14.44940442175313
119200000 14.518534971906583
119300000 14.469619352579006
119400000 14.060680146977072
119500000 12.880995707870989
119600000 12.889617940948666
119700000 12.976558903097981
119800000 12.770663223123542
119900000 13.28847176412862
120000000 12.523250592719576
120100000 5.959523571304372
120200000 11.868204312744977
120300000 5.352082199845317
120400000 11.767843996089548
120500000 7.457189840802159
120600000 11.35247062402462
120700000 12.704447437007317
120800000 12.969409358934728
120900000 12.876674889490568
121000000 12.601873475384146
121100000 13.115922149035123
121200000 12.843254053526286
121300000 14.28755074924546

...

175500000 10.287722387883745
175600000 9.296539323640951
175700000 9.377154981718974
175800000 9.044862445769432
175900000 9.253314704348933
176000000 9.119483397712026
176100000 8.610472062719259
176200000 9.138268068345248
176300000 3.852681813915991
176400000 8.862225000407166
176500000 3.8322229680523274
176600000 7.455912285138398
176700000 6.120749427098768
176800000 6.197517450443804
176900000 9.30755299829538
177000000 8.713457701819095
177100000 8.89155707536772
177200000 9.181840837231725
177300000 8.779105370968349
177400000 9.779399623988171
177500000 9.690669842103715
177600000 10.048190580636717
177700000 9.612188021473955
177800000 9.647963123243905
177900000 9.295503628539688
178000000 8.655881425404937
178100000 8.971863483613499
178200000 8.825193830664068
178300000 8.583169585147113
178400000 8.691138265448386
178500000 5.41879876238228
178600000 8.139391220645727
178700000 7.668419193962378
178800000 4.294510571954978
178900000 8.493173684292078
179000000 3.83090020

204900000 7.054477646086988
205000000 7.071324837545424
205100000 6.913611614968833
205200000 6.845961112930135
205300000 2.8676384244009356
205400000 6.970845704704381
205500000 6.614199564264791
205600000 6.72458403609408
205700000 4.443562561436882
205800000 4.718693491571152
205900000 6.710068598740609
206000000 6.792069205560432
206100000 6.842968219094362
206200000 2.4915056359743777
206300000 7.05182086061219
206400000 7.051629133003307
206500000 7.196257782728395
206600000 7.3651418914302695
206700000 7.475801374381488
206800000 7.595413436413258
206900000 7.218667033259474
207000000 7.096958280314961
207100000 7.098059840367754
207200000 6.6878647731659155
207300000 6.689753879719007
207400000 6.464542790341776
207500000 4.057681909453167
207600000 5.578486898942619
207700000 6.251515099325091
207800000 6.320179509111637
207900000 6.440580916203194
208000000 1.8432392398245832
208100000 6.691158100169433
208200000 6.205490361656792
208300000 6.364188609771829
208400000 4.93396

234100000 4.886472139699131
234200000 4.984079759610067
234300000 1.9316210580456321
234400000 4.798115457970917
234500000 4.559107075097438
234600000 4.3310997201748025
234700000 4.393702172407955
234800000 4.296416851404448
234900000 3.9181873127699696
235000000 2.107588838317474
235100000 4.292902278123945
235200000 4.333585607520322
235300000 4.2026941795689865
235400000 4.567773394590288
235500000 4.505279191374775
235600000 3.141264082084536
235700000 3.5561691916029847
235800000 5.065945421134882
235900000 5.00551737751629
236000000 4.95887046427644
236100000 5.454973375733256
236200000 5.178052195308546
236300000 5.2118876150464555
236400000 5.276422586880197
236500000 3.0107970436960163
236600000 3.8891107049482034
236700000 5.096371668451196
236800000 4.481131403808236
236900000 4.6317791167656575
237000000 4.1567162448318005
237100000 4.5560665473008255
237200000 2.015033298249655
237300000 3.9366121336211095
237400000 4.663596350838661
237500000 4.266116251561397
237600000 

263100000 2.7689388193814723
263200000 2.9058634239640333
263300000 1.14202113868117
263400000 3.0565292957608827
263500000 2.8171885368620755
263600000 2.713148552669555
263700000 2.9247949761442813
263800000 2.516949160196662
263900000 2.8147258054139614
264000000 2.528997519666125
264100000 2.6157947220565476
264200000 0.7433964299372965
264300000 2.588812934035066
264400000 2.7083822204138337
264500000 2.613647796612591
264600000 2.7803565323492085
264700000 2.771879655944487
264800000 2.70327362045653
264900000 2.8718883794978227
265000000 2.730726562772045
265100000 0.9690247723058893
265200000 2.6675695184449952
265300000 2.7461962726312277
265400000 2.639852511210412
265500000 1.581943830746707
265600000 2.183569901389182
265700000 2.478413994655977
265800000 2.705413359327008
265900000 2.443154236696849
266000000 2.4964442497904167
266100000 2.488546668966386
266200000 2.345499057065338
266300000 2.3653000807422266
266400000 1.5055774164691733
266500000 1.6161393238081039
2666

291600000 0.27044364765977197
291700000 0.26897203549129767
291800000 0.3199139355164104
291900000 0.2690698100646602
292000000 0.21882432270678878
292100000 0.22940930809940566
292200000 0.2557396523827281
292300000 0.0441976434302926
292400000 0.20679112705415487
292500000 0.24854303168285885
292600000 0.22672557302586235
292700000 0.24003998116833633
292800000 0.23390991889587373
292900000 0.17368188403352103
293000000 0.13501753601716626
293100000 0.13359159185675115
293200000 0.13998783609008789
293300000 0.16582623032914928
293400000 0.10402346710855762
293500000 0.11074510214544336
293600000 0.14515538175223602
293700000 0.1485883656833205
293800000 0.1207674291311105
293900000 0.10505248883396719
294000000 0.13537114054511323
294100000 0.11484185962566733
294200000 0.14708505107001132
294300000 0.13402110565429262
294400000 0.10662309682966271
best so far: 0
type: [1, 1, 1, 131, 1] 131
cases of this type: 2248091
294500000 0.053158336407320365
294600000 0.008817950654731857
294